# 필요한 패키지 다운로드

In [ ]:
!pip install pdblp

In [1]:
# 블룸버그 터미널 관련
import pdblp

# 병렬 연산 관련
from joblib import Parallel, delayed

# 데이터셋 전처리
import web_Dataset_Preprocessing as D_P

# 블룸버그 터미널 연결

In [3]:
con = pdblp.BCon(debug = False, port=8194, timeout=5000)
    # Bloomberg 서비스에 연결하기 위한 BCon 객체 생성
    # debug=False는 디버그 모드를 비활성화하는 옵션, 디버그 모드를 활성화하면 더 자세한 정보를 얻을 수 있지만, 일반적으로는 비활성화하는 것이 좋음
    # port=8194는 Bloomberg 서비스에 연결하기 위한 포트 번호, 기본 포트 번호는 8194
    # timeout=5000은 연결 시도를 중지하고 예외를 발생시킬 최대 대기 시간(밀리초), 5000밀리초(5초)로 설정되어 있으므로, 연결 시도가 5초 동안 성공하지 않으면 예외가 발생

con.start()

pdblp.pdblp:WARNING:Message Received:
SessionStartupFailure = {
    reason = {
        source = "ApiConnector"
        category = "IO_ERROR"
        errorCode = 8
        description = "connect event failed"
    }
}



ConnectionError: Could not start blpapi.Session

# 초기 bulk dataset 만들기 

In [1]:
kospi_russell_members = []

⬇️ (차후 개선) 2500개에서 짤리는 거 방지하는 코드 추가

In [ ]:
russell_members_df = con.bulkref('RAY Index', 'INDX_MEMBERS')
russell_members = rusell_members_df[['value']]
kospi_russell_members.append(list(russell_members))

➡️ (차후 개선) russell 및 코스피 members 저장해놓는 db 만들기

In [ ]:
kospi_russell_tickers = [x + ' Equity' for x in kospi_russell_members]

⬇️ (차후 개선) 시작일 협의 + 최신일 수정

⬇️ (차후 개선) 'VOLUME' 넣을지 협의 -> 전처리 코드도 수정 필요함

In [ ]:
con.bdh(ticker, ['PX_LAST', 'CHG_PCT_1D'], '20150101', '20230717')

⬇️ (차후 개선) bloomberg랑 병렬 연산 같이 되는지 확인 필요

In [ ]:
for ticker in kospi_russell_tickers:
    
    # 블룸버그에서 raw data 받아오기
    if '/' in ticker:
        ticker = ticker.replace("/", "-")
    ticker_bloomgberg_df = con.bdh(ticker, ['PX_LAST', 'CHG_PCT_1D'], '20150101', '20230717')
    
    # 데이터 전처리
    

In [9]:
import pandas as pd

In [10]:
df_test = pd.read_csv('/Users/chanj/OneDrive/바탕 화면/Life/datasets-total-ver23060812/datasets-KOSPI200/dataset-KOSPI200-000100 KS.csv')

In [11]:
df_test.head()

,date,PX_OPEN,PX_LOW,PX_HIGH,PX_LAST,VOLUME,MOV_AVG_5D,CHG_PCT_1D
0,2015-01-02,23950.0,23950.0,24305.0,24021.0,93579.0,24276.8742,-0.2950
1,2015-01-05,23879.0,23808.0,24163.0,23879.0,146986.0,24191.5925,-0.5917
2,2015-01-06,23879.0,23737.0,24092.0,23737.0,102050.0,24035.2426,-0.5952
3,2015-01-07,23950.0,23524.0,23950.0,23524.0,140731.0,23850.4654,-0.8982
4,2015-01-08,23595.0,23097.0,23737.0,23310.0,329698.0,23694.1155,-0.9063


In [12]:
df_test2=df_test[['date', 'PX_LAST', 'CHG_PCT_1D']]

In [13]:
import pandas as pd
import numpy as np

from scipy.stats.mstats import winsorize

from datetime import datetime
import pandas_market_calendars as mcal

In [14]:
df_test3 = D_P.preprocessing(df_test2)

C:\Users\chanj\OneDrive\바탕 화면\life_web_ui\web_Dataset_Preprocessing.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df['date'] = pd.to_datetime(self.df['date'])
C:\Users\chanj\OneDrive\바탕 화면\life_web_ui\web_Dataset_Preprocessing.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df['LOG_CHG_PCT_1D'] = np.log(self.df['CHG_PCT_1D']/100 + 1)
C:\Users\chanj\anaconda3\lib\site-packages\pandas_market_calendars\market_calendar.py:144: UserWarning: ['break_start', 'break_end'] are discontinued, the

TypeError: 'NoneType' object is not subscriptable